# Capstone Project - League of Legends Champion Recommender

> Author: Ryan Yong

**Summary:**   
- Develop a Recommender System for recommending champions to users based on their account mastery points.
- Training data: Account & Champion Data

There are a total of 7 notebooks for this project:  
 1. `01a_data_scrape.ipynb`   
 2. `01b_wiki_scrape_fail.ipynb`   
 3. `02_champion_dataset_EDA.ipynb`
 4. `03_account_dataset_EDA.ipynb`
 5. `04_intial_recommender_system.ipynb`
 6. `05_final_hybrid_system.ipynb`
 7. `06_implementation.ipynb`

---
**This Notebook**
- shows the initial attempt at building a recommender system based on consine similarity
- the inital attempt does not use `champion_data` at all, but solely based on user similarity

## Recommender model using Cosine Similarity

In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Function: `recommend_champions`

This function recommends champions for a user based on their mastery scores, the rank of players in the dataset, and optional weighting by rank similarities.

#### Parameters:
- `user_data`: DataFrame containing user's data including mastery scores and rank.
- `user_index`: Index of the specific user in `user_data` to make recommendations for.
- `consider_ranks`: Boolean flag to determine if filtering of the main data should be based on user rank; set to `False` by default.
- `weight_by_rank`: Boolean flag to decide if the similarity scores should be adjusted based on the rank differences; set to `False` by default.

#### Process:

1. **Initialization and Data Loading**:
   - Retrieves the user's rank.
   - Initializes a MinMaxScaler for normalizing data.
   - Loads the main dataset from a specified path.

2. **Data Filtering**:
   - If `consider_ranks` is `True`, filters out players in the main dataset who have a rank lower than the user's rank.

3. **Data Preprocessing**:
   - Uses a nested function `preprocess_main_data` to log-transform and scale the main data (excluding the 'rank' column).
   - Another nested function `transform_user_data` is used to transform the user data by log-transforming and scaling using the same scaler.

4. **Similarity Calculation**:
   - Computes the cosine similarity between the user's data and the preprocessed main dataset.
   - Extracts similarity scores for the specific user.

5. **Weight Adjustment by Rank**:
   - If `weight_by_rank` is enabled, adjusts the similarity scores based on the rank differences. This includes:
     - Calculating primary weights based on relative ranks.
     - Calculating secondary weights based on the exponential decay of rank differences.
     - Multiplying these weights with the similarity scores to get weighted scores.

6. **Recommendation Generation**:
   - Identifies the top 5 similar users based on the weighted/unweighted similarity scores.
   - Aggregates the mastery levels of these users to get an average mastery profile.
   - Filters out champions that the user has already extensively played, based on a threshold.

7. **Output**:
   - Returns the top 3 recommended champions sorted by their potential interest level, which is indicated by the average mastery level of similar users.

#### Usage:
This function is useful for building personalized champion recommendations in games, particularly when trying to suggest new options based on user behavior and preferences of similarly ranked players.


In [24]:
# Function to recommend champions, using the preprocessed main data
def recommend_champions(user_data, user_index, consider_ranks=False, weight_by_rank=False):
    user_rank = user_data.loc[user_index, 'rank']
   
    # Initialize the scaler
    scaler = MinMaxScaler()

    # Function to preprocess the main data by log-transforming and scaling
    def preprocess_main_data(df):
        df_transformed = np.log1p(df.drop(columns=['rank']))
        return pd.DataFrame(scaler.fit_transform(df_transformed), columns=df_transformed.columns)

    # Function to transform the user data by log-transforming and using the existing scale
    def transform_user_data(df):
        df_transformed = np.log1p(df.drop(columns=['rank']))
        return pd.DataFrame(scaler.transform(df_transformed), columns=df_transformed.columns)

    data_path = '../data/account_rank_mastery_dataset.csv'  # Adjust path as needed
    df = pd.read_csv(data_path, index_col=False)
   

    # Optionally filter data to only include players of equal or higher rank
    if consider_ranks and user_rank != 0:
        df = df[df['rank'] >= user_rank]

    # Preprocess the main dataset (excluding 'rank' column)
    data_preprocessed = preprocess_main_data(df)

    # Process user data and exclude the rank column
    user_mastery_data = transform_user_data(user_data.iloc[[user_index]])

    # Calculate the similarity matrix between the user and the main data
    similarity_matrix = cosine_similarity(user_mastery_data, data_preprocessed)

    # Get the similarity scores for the user
    user_similarity_scores = similarity_matrix[0]

    # Modify similarity scores by rank if needed
    if weight_by_rank and user_rank != 0:
        ranks = df['rank']
        primary_weights = (ranks - user_rank) / 10.0 + 1
        secondary_weights = np.exp(-np.abs(ranks - user_rank) / 10.0)
        total_weights = primary_weights * secondary_weights
        user_similarity_scores *= total_weights

    # Get the indices of the top 5 similar users
    top_similar_indices = np.argsort(-user_similarity_scores)[:5]

    # Aggregate the mastery levels of these top similar users
    recommendations = data_preprocessed.iloc[top_similar_indices].mean(axis=0)

    # Filter out champions the user has already played enough
    recommended_champions = recommendations[(recommendations < 0.2) & (user_mastery_data < 0.2).iloc[0]]

    # Return the top 3 recommended champions sorted by their average mastery level in descending order
    return recommended_champions.sort_values(ascending=False).head(3).index.tolist()


In [25]:
# Example usage:
users_data_path = '../data/test_accounts_rank_mastery.csv'  # Modify with the actual path to your users_df
users_df = pd.read_csv(users_data_path,index_col= False)
users_df_without_name = users_df.drop(columns='user_name')




Recommended Champions for Dorainen#FKKK at index 5: ['Milio']


In [27]:
users_df.head(12)

,user_name,rank,Aatrox,Ahri,Akali,Akshan,Alistar,Amumu,Anivia,Annie,Aphelios,Ashe,Aurelion Sol,Azir,Bard,Bel'Veth,Blitzcrank,Brand,Braum,Briar,Caitlyn,Camille,Cassiopeia,Cho'Gath,Corki,Darius,Diana,Dr. Mundo,Draven,Ekko,Elise,Evelynn,Ezreal,Fiddlesticks,Fiora,Fizz,Galio,Gangplank,Garen,Gnar,Gragas,Graves,Gwen,Hecarim,Heimerdinger,Hwei,Illaoi,Irelia,Ivern,Janna,Jarvan IV,Jax,Jayce,Jhin,Jinx,K'Sante,Kai'Sa,Kalista,Karma,Karthus,Kassadin,Katarina,Kayle,Kayn,Kennen,Kha'Zix,Kindred,Kled,Kog'Maw,LeBlanc,Lee Sin,Leona,Lillia,Lissandra,Lucian,Lulu,Lux,Malphite,Malzahar,Maokai,Master Yi,Milio,Miss Fortune,Mordekaiser,Morgana,Naafiri,Nami,Nasus,Nautilus,Neeko,Nidalee,Nilah,Nocturne,Nunu & Willump,Olaf,Orianna,Ornn,Pantheon,Poppy,Pyke,Qiyana,Quinn,Rakan,Rammus,Rek'Sai,Rell,Renata Glasc,Renekton,Rengar,Riven,Rumble,Ryze,Samira,Sejuani,Senna,Seraphine,Sett,Shaco,Shen,Shyvana,Singed,Sion,Sivir,Skarner,Smolder,Sona,Soraka,Swain,Sylas,Syndra,Tahm Kench,Taliyah,Talon,Taric,Teemo,Thresh,Tristana,Trundle,Tryndamere,Twisted Fate,Twitch,Udyr,Urgot,Varus,Vayne,Veigar,Vel'Koz,Vex,Vi,Viego,Viktor,Vladimir,Volibear,Warwick,Wukong,Xayah,Xerath,Xin Zhao,Yasuo,Yone,Yorick,Yuumi,Zac,Zed,Zeri,Ziggs,Zilean,Zoe,Zyra
0,Atrophy#Fiend,7,5160.0,29543.0,20680.0,444.0,20952.0,13173,9825.0,19599,31661.0,358466.0,3370.0,27309.0,80978.0,0.0,59672.0,10509.0,10503.0,0.0,683542.0,12646.0,6493.0,8886.0,10705.0,18573.0,2871.0,9451.0,48041.0,26354.0,5915.0,3303.0,240035,2985.0,57646.0,24507.0,3283.0,53895.0,23216.0,16103.0,36344.0,58245.0,0.0,39768.0,4211.0,133.0,25147.0,87371.0,551.0,18731.0,14767.0,24395.0,77139.0,126629.0,89598.0,0.0,247099.0,28691.0,25193.0,23350.0,2425.0,19304.0,17265.0,9021.0,22216.0,30135.0,11191.0,425.0,75801.0,8039.0,325494.0,16884.0,0.0,5732.0,172077.0,12028.0,17950,33320,3336.0,15019,9335,0.0,208561.0,29544,3121.0,0.0,2932.0,19385.0,10106,7061.0,102278.0,6629.0,3157.0,6200,19328.0,77183.0,8313.0,4704.0,13787.0,19825.0,11392.0,7680.0,8434.0,10007.0,12986.0,2816.0,5169.0,13270.0,32861.0,259217.0,17826.0,12970.0,6474.0,4317.0,111057.0,25550.0,43410.0,9730.0,23668.0,16093.0,11364.0,7784.0,136649.0,3777.0,5690.0,4523.0,7294.0,233260.0,9049.0,29223.0,3300.0,5536.0,7597.0,13740.0,2747.0,196786,50672,10831.0,8667.0,45727.0,23902.0,23315.0,2603.0,71535.0,201080,39527.0,5566,865.0,1197.0,105.0,10590.0,6623.0,4081.0,2635.0,5649.0,109073.0,7076.0,6967.0,125054,4193.0,1597.0,3825.0,30053.0,113469.0,64129.0,66710.0,2373.0,4880.0,14277.0
1,Aradia#NaCl,5,0.0,149153.0,4574.0,0.0,179.0,5389,4364.0,15647,4857.0,52300.0,14121.0,4528.0,31912.0,0.0,136878.0,29549.0,43445.0,0.0,67287.0,0.0,5349.0,3015.0,196.0,7846.0,0.0,284.0,6193.0,12600.0,7460.0,5739.0,134905,7454.0,1944.0,14745.0,0.0,4498.0,10444.0,0.0,4931.0,2254.0,0.0,1090.0,2783.0,772.0,0.0,1253.0,1269.0,152181.0,4066.0,1067.0,2120.0,49798.0,631983.0,0.0,2022.0,7074.0,16719.0,8338.0,1817.0,26522.0,4425.0,0.0,2183.0,5897.0,1252.0,1440.0,16524.0,168101.0,12385.0,26266.0,0.0,2332.0,184231.0,92047.0,461657,10823,7421.0,2843,4187,8674.0,38910.0,5936,301903.0,0.0,247064.0,3222.0,168924,4490.0,123477.0,0.0,434.0,1065,776.0,153887.0,532.0,350.0,3282.0,87987.0,1354.0,0.0,146808.0,1734.0,0.0,1226.0,754.0,0.0,0.0,39951.0,0.0,7182.0,0.0,1116.0,31269.0,2218.0,1210.0,31285.0,6888.0,0.0,150.0,901.0,35078.0,0.0,0.0,179535.0,148865.0,11261.0,6667.0,79348.0,9505.0,2389.0,0.0,63664.0,10775.0,515447,10187,908.0,0.0,8138.0,6539.0,0.0,2933.0,21310.0,117580,42095.0,15647,1137.0,0.0,0.0,1563.0,17333.0,0.0,0.0,0.0,31414.0,107372.0,98.0,9816,0.0,0.0,46968.0,121.0,20055.0,4997.0,38670.0,34032.0,54496.0,12259.0
2,Commet#OG1,7,78181.0,36448.0,17277.0,0.0,7847.0,159080,114295.0,65011,0.0,22185.0,17726.0,5925.0,37229.0,167290.0,8844.0,2336.0,10258.0,31111.0,4730.0,15425.0,14607.0,74153.0,0.0,109695.0,27678.0,20249.0,35543.0,181633.0,28709.0,47229.0,119005,45320.0,5218.0,30871.0,16481.0,3030.0,69042.0,49714.0,40503.0,4861.0,0.0,99266.0,13594.0,786.0,37775.0,18702.0,0.0,0.0,24487.0,13938.0,30533.0,154331.0,116454.0,207.0,83919.0,1240.0,8254.0,2

In [31]:
user_index = 3  # Index of the user in users_df to be recommended champions
user_name = users_df['user_name'].loc[user_index-1]
recommended_champions = recommend_champions( users_df_without_name, user_index)
print(f"Recommended Champions for {user_name} at index {user_index}:", recommended_champions)

Recommended Champions for Commet#OG1 at index 3: ['Zoe', 'Cassiopeia', 'Nami']


In [32]:
user_index = 5  # Index of the user in users_df to be recommended champions
user_name = users_df['user_name'].loc[user_index-1]
recommended_champions = recommend_champions( users_df_without_name, user_index)
print(f"Recommended Champions for {user_name} at index {user_index}:", recommended_champions)

Recommended Champions for Clairvoyant#8721 at index 5: ['Milio']


In [33]:
user_index = 6 # Index of the user in users_df to be recommended champions
user_name = users_df['user_name'].loc[user_index-1]
recommended_champions = recommend_champions( users_df_without_name, user_index)
print(f"Recommended Champions for {user_name} at index {user_index}:", recommended_champions)

Recommended Champions for Dorainen#FKKK at index 6: ['Vex', 'Rell', 'Renata Glasc']


## Conclusion

As shown above, there are some issues with the recommender system. 

1. The recommendations are based on a difference between similar users and the targeted user, meaning that champions that the target user plays on a regular basis but less than the similar users will still be recommended.

2. There is no objective metric to evaluate the recommender system at the moment

3. Some users like Clarivoyant seem to get less than the expected 3 recommendations as none of the other champions managed to fit the criteria